In [1]:
import pandas as pd
from __future__ import absolute_import, division

import os
import time
import numpy as np
import pandas as pd
import gensim
from tqdm import tqdm
import random
import psutil
from multiprocessing import Pool
import multiprocessing as mp
from gensim.models import KeyedVectors
import re
import gc
import pickle
import operator

In [2]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    embeddings_index = {}
    f = open(path,encoding='utf8')
    print(f)
    for line in f:
        values = line.split()
        word = ' '.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs.reshape(-1)
    f.close()
    return embeddings_index

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
#     sentences=str(sentences)
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return a,sorted_x

import string
def text_to_wordlist(text, remove_stopwords=False,stem_words=False,lemma_word=False):
    
    text = str(text)
    text = text.lower().split()
    
    #remove stops
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        
    text = " ".join(text)
    
    text = re.sub(r",",r"",text)
    text = re.sub(u'\xa0', u' ',text)
    text=re.sub(r"\'",r"'",text)
    text = re.sub(r" e.g ", " eg ", text)
    text = re.sub(r" b.g ", " bg ", text)
    text = re.sub(r"U.S", "US", text)
    text = re.sub(r"sharethefacts{.*?Share The Facts"," ",text)
    
    # stemmer
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
        
#     text = text.lower()
        
    return text


In [3]:
GLOVE_EMBEDDING_PATH= "../glove/glove6B/glove.6B.300d.txt"
num_models = 2
lstm_units = 128
dense_hidden_units = 4 * lstm_units
max_len = 220
embed_size = 300
max_features = 120000

tic = time.time()
glove_embeddings = load_embeddings(GLOVE_EMBEDDING_PATH)
print(f'loaded {len(glove_embeddings)} word vectors in {time.time()-tic}s')

# %%time
# load = True
# if not load:
names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party",
                                            "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue","justifikasi"]
    
train= pd.read_csv("../LIAR-PLUS/dataset/train.tsv",delimiter='\t',header=None, error_bad_lines=False,names=names)
test= pd.read_csv("../LIAR-PLUS/dataset/test.tsv",delimiter='\t',header=None, error_bad_lines=False,names=names)
valid= pd.read_csv("../LIAR-PLUS/dataset/valid.tsv",delimiter='\t',header=None, error_bad_lines=False,names=names)
train.dropna(subset=['justifikasi'], how='all', inplace = True)
test.dropna(subset=['justifikasi'], how='all', inplace = True)
# train.dropna()

<_io.TextIOWrapper name='../glove/glove6B/glove.6B.300d.txt' mode='r' encoding='utf8'>
loaded 400001 word vectors in 24.75396227836609s


In [4]:
import string
from nltk.tokenize.treebank import TreebankWordTokenizer

data_train = train['justifikasi']

latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
white_list = string.ascii_letters + string.digits + latin_similar + ' '
white_list += "'"

glove_chars = ''.join([c for c in tqdm(glove_embeddings) if len(c) == 1])
glove_symbols = ''.join([c for c in glove_chars if not c in white_list])
print(glove_symbols)
jigsaw_chars = build_vocab(list(data_train))
jigsaw_symbols = ''.join([c for c in jigsaw_chars if not c in white_list])
print(jigsaw_symbols)
symbols_to_delete = ''.join([c for c in jigsaw_symbols if not c in glove_symbols])
print(symbols_to_delete)
symbols_to_isolate = ''.join([c for c in jigsaw_symbols if c in glove_symbols])
print(symbols_to_isolate)


isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x=str(x)
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x


def handle_contractions(x):
    tokenizer = TreebankWordTokenizer()
    x = tokenizer.tokenize(x)
    x = ' '.join(x)
    return x

def preprocess(x):
    x = text_to_wordlist(x)
    x = handle_punctuation(x)
    x = handle_contractions(x)
#     x = x.lower()
    return x

data_train = train['justifikasi'].apply(lambda x: preprocess(x))
data_test = test['justifikasi'].apply(lambda x: preprocess(x))
data_valid = valid['justifikasi'].apply(lambda x: preprocess(x))


 26%|██▌       | 2678/10205 [00:00<00:00, 26773.71it/s]

,."-():$;_?–%/!`&=—“”#[+¥£|]~\{…½>€}*@<»«、α¼^βφωλσδγθπ„με¾¡ρ¢ψτηƒκ₤⅓иνζ¿χξ⅔в―ο。‚сιರа⅛རསυя›ಕངоو‹уನ˚ದනгරאеತಸകಮ⅜ག₂ව·ಪල¤ὁදкසವಗіរ⅝කལϕནདಲтಯಡಬ¨ъടම‟мಶь⅞ה镇පයнªམ乡තлಟབпសಹ道ɔରതф￥ר王хἡчэзപʃḳшיוಳបʔកбדග⁰дಜש李жєבតס神姬人一ю


100%|██████████| 10205/10205 [00:00<00:00, 27276.46it/s]


.-,"…()“”;$?:— –{#%}_@/* ‪‎‬&!
	•[]+=`½➤\­― ⅔
  ‪‎‬
	•➤­ 
.-,"…()“”;$?:—–{#%}_@/*&![]+=`½\―⅔


In [12]:
data_test

0       meantime engineering experts agree the wall wo...
1       she cited layoff notices received by the state...
2       trump said that mccain `` has done nothing to ...
3       but spending still goes up . in addition many ...
4       our rating a democratic party web video making...
5       rhode island ’ s former top cop is setting his...
6       residents in those suburbs do not get a vote b...
7       boehner may be technically correct that biden ...
8       speaking at the democratic national convention...
9       pence said `` we know that more than half of h...
10      there ’ s little concrete evidence that defini...
11      we informed mackowiak of the date conflict . h...
12      the word `` denali `` doesn ’ t show up in swa...
13      a social - media meme says that 57 percent of ...
14      he clearly has some misgivings about residency...
15      in october it was 4 . 4 percent according to t...
16      as senate president niehaus is one of the most...
17      blackb

In [5]:
vocab = build_vocab(list(data_train.apply(lambda x:x.lower().split())))
known,unknown = check_coverage(vocab,glove_embeddings)
print(unknown[:10])
print(known.keys())

100%|██████████| 22886/22886 [00:00<00:00, 1344183.63it/s]

Found embeddings for 93.60% of vocab
Found embeddings for  99.54% of all text
[('sharethefacts', 556), ('0px', 143), ('politifactri', 134), ('rgba', 99), ('2px', 76), ('24px', 65), ('14px', 63), ('2196f3', 55), ('32px', 54), ('20px', 50)]
dict_keys(['that', "'s", 'a', 'premise', 'he', 'fails', 'to', 'back', 'up', '.', 'annie', 'list', 'makes', 'no', 'bones', 'about', 'being', 'comfortable', 'with', 'candidates', 'who', 'oppose', 'further', 'restrictions', 'on', 'late', '-', 'term', 'abortions', 'then', 'again', 'this', 'year', 'it', '’', 's', 'backing', 'two', 'house', 'voted', 'for', 'more', 'limits', 'said', 'the', 'decline', 'of', 'coal', '``', 'started', 'when', 'natural', 'gas', 'took', 'off', '…', 'begin', 'in', 'president', '(', 'george', 'w', ')', 'bush', 'administration', 'doubt', 'has', 'been', 'gaining', 'ground', 'generating', 'electricity', 'trend', '1990s', 'but', 'clearly', 'gained', 'speed', 'during', 'production', 'competitor', 'picked', 'analysts', 'give', 'little', '

In [6]:
vocab2 = build_vocab(list(data_test.apply(lambda x:x.lower().split())))
oov2 = check_coverage(vocab2,glove_embeddings)
oov2[:10]


100%|██████████| 9255/9255 [00:00<00:00, 1113132.90it/s]


Found embeddings for 97.96% of vocab
Found embeddings for  99.77% of all text


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [7]:
vocab3 = build_vocab(list(data_valid.apply(lambda x:x.lower().split())))
oov3 = check_coverage(vocab3,glove_embeddings)
oov3[:10]



100%|██████████| 9076/9076 [00:00<00:00, 1125458.35it/s]

Found embeddings for 97.69% of vocab
Found embeddings for  99.73% of all text



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [10]:
#####################
###   POS TAGS    ###
#####################

import stanfordnlp
import pandas as pd
import pickle
# import data_util as du

# train,test,valid = du.load_dataset()
nlp = stanfordnlp.Pipeline(processors = "tokenize,mwt,lemma,pos")


Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/hdd1/rini.jannati/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/hdd1/rini.jannati/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: pos
With settings: 
{'model_path': '/hdd1/rini.jannati/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/hdd1/rini.jannati/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


In [11]:
def get_word(doc):
    doc = nlp(doc)
#     print(doc)
    word=list()
    for sent in doc.sentences:
        for wrd in sent.words:
            print(wrd.text," POSTag: ",wrd.upos)
            word.append(wrd)
#             print(type(wrd))
    return word

nlpst_test_ = data_test.apply(lambda x: get_word(x))



meantime  POSTag:  NOUN
engineering  POSTag:  NOUN
experts  POSTag:  NOUN
agree  POSTag:  VERB
the  POSTag:  DET
wall  POSTag:  NOUN
would  POSTag:  AUX
most  POSTag:  ADV
likely  POSTag:  ADV
take  POSTag:  VERB
years  POSTag:  NOUN
to  POSTag:  PART
complete  POSTag:  VERB
.  POSTag:  PUNCT
keep  POSTag:  VERB
in  POSTag:  ADP
mind  POSTag:  NOUN
too  POSTag:  ADV
it  POSTag:  PRON
took  POSTag:  VERB
more  POSTag:  ADJ
than  POSTag:  ADP
six  POSTag:  NUM
years  POSTag:  NOUN
to  POSTag:  PART
build  POSTag:  VERB
roughly  POSTag:  ADV
700  POSTag:  NUM
miles  POSTag:  NOUN
of  POSTag:  ADP
fence  POSTag:  NOUN
and  POSTag:  CCONJ
barriers  POSTag:  NOUN
along  POSTag:  ADP
the  POSTag:  DET
roughly  POSTag:  ADV
2000  POSTag:  NUM
-  POSTag:  PUNCT
mile  POSTag:  NOUN
u  POSTag:  PRON
.  POSTag:  PUNCT
s  POSTag:  PART
.  POSTag:  PUNCT
-  POSTag:  PUNCT
mexico  POSTag:  PROPN
border  POSTag:  NOUN
.  POSTag:  PUNCT
she  POSTag:  PRON
cited  POSTag:  VERB
layoff  POSTag:  NOUN
noti

residents  POSTag:  NOUN
in  POSTag:  ADP
those  POSTag:  DET
suburbs  POSTag:  NOUN
do  POSTag:  AUX
not  POSTag:  PART
get  POSTag:  VERB
a  POSTag:  DET
vote  POSTag:  NOUN
because  POSTag:  SCONJ
they  POSTag:  PRON
already  POSTag:  ADV
have  POSTag:  AUX
authorized  POSTag:  VERB
sales  POSTag:  NOUN
tax  POSTag:  NOUN
increases  POSTag:  NOUN
for  POSTag:  ADP
their  POSTag:  PRON
towns  POSTag:  NOUN
.  POSTag:  PUNCT
another  POSTag:  DET
argument  POSTag:  NOUN
by  POSTag:  ADP
proponents  POSTag:  NOUN
is  POSTag:  VERB
that  POSTag:  SCONJ
a  POSTag:  DET
vote  POSTag:  NOUN
against  POSTag:  ADP
the  POSTag:  DET
half  POSTag:  NOUN
-  POSTag:  PUNCT
ce  POSTag:  NOUN
nt  POSTag:  PART
sales  POSTag:  NOUN
tax  POSTag:  NOUN
increase  POSTag:  NOUN
is  POSTag:  AUX
essentially  POSTag:  ADV
a  POSTag:  DET
vote  POSTag:  NOUN
for  POSTag:  ADP
a  POSTag:  DET
property  POSTag:  NOUN
-  POSTag:  PUNCT
tax  POSTag:  NOUN
increase  POSTag:  NOUN
to  POSTag:  PART
fund  POSTag

the  POSTag:  DET
word  POSTag:  NOUN
``  POSTag:  PUNCT
denali  POSTag:  PROPN
``  POSTag:  PUNCT
doesn  POSTag:  VERB
’  POSTag:  PUNCT
t  POSTag:  NOUN
show  POSTag:  VERB
up  POSTag:  ADP
in  POSTag:  ADP
swahili  POSTag:  ADJ
one  POSTag:  NUM
of  POSTag:  ADP
kenya  POSTag:  PROPN
’  POSTag:  PUNCT
s  POSTag:  PART
two  POSTag:  NUM
national  POSTag:  ADJ
languages  POSTag:  NOUN
.  POSTag:  PUNCT
instead  POSTag:  ADV
it  POSTag:  PRON
’  POSTag:  PUNCT
s  POSTag:  AUX
koyukon  POSTag:  ADJ
athabaskan  POSTag:  NOUN
for  POSTag:  ADP
``  POSTag:  PUNCT
high  POSTag:  ADJ
``  POSTag:  PUNCT
or  POSTag:  CCONJ
``  POSTag:  PUNCT
tall  POSTag:  ADJ
.  POSTag:  PUNCT
``  POSTag:  PUNCT
a  POSTag:  DET
social  POSTag:  ADJ
-  POSTag:  PUNCT
media  POSTag:  NOUN
meme  POSTag:  NOUN
says  POSTag:  VERB
that  POSTag:  SCONJ
57  POSTag:  NUM
percent  POSTag:  NOUN
of  POSTag:  ADP
federal  POSTag:  ADJ
spending  POSTag:  NOUN
goes  POSTag:  VERB
to  POSTag:  ADP
the  POSTag:  DET
militar

it  POSTag:  PRON
was  POSTag:  AUX
granted  POSTag:  VERB
a  POSTag:  DET
charter  POSTag:  NOUN
by  POSTag:  ADP
congress  POSTag:  PROPN
in  POSTag:  ADP
1863  POSTag:  NUM
and  POSTag:  CCONJ
has  POSTag:  VERB
a  POSTag:  DET
mandate  POSTag:  NOUN
to  POSTag:  PART
advise  POSTag:  VERB
the  POSTag:  DET
federal  POSTag:  ADJ
government  POSTag:  NOUN
on  POSTag:  ADP
scientific  POSTag:  ADJ
and  POSTag:  CCONJ
technical  POSTag:  ADJ
matters  POSTag:  NOUN
.  POSTag:  PUNCT
the  POSTag:  DET
report  POSTag:  NOUN
found  POSTag:  VERB
that  POSTag:  SCONJ
people  POSTag:  NOUN
without  POSTag:  ADP
health  POSTag:  NOUN
care  POSTag:  NOUN
coverage  POSTag:  NOUN
:  POSTag:  PUNCT
received  POSTag:  VERB
less  POSTag:  ADV
frequent  POSTag:  ADJ
or  POSTag:  CCONJ
no  POSTag:  DET
use  POSTag:  NOUN
of  POSTag:  ADP
cancer  POSTag:  NOUN
screening  POSTag:  NOUN
tests  POSTag:  NOUN
;  POSTag:  PUNCT
received  POSTag:  VERB
inconsistent  POSTag:  ADJ
care  POSTag:  NOUN
for  POS

our  POSTag:  PRON
job  POSTag:  NOUN
as  POSTag:  ADP
fact  POSTag:  NOUN
-  POSTag:  PUNCT
checkers  POSTag:  NOUN
in  POSTag:  ADP
this  POSTag:  DET
case  POSTag:  NOUN
is  POSTag:  VERB
not  POSTag:  PART
to  POSTag:  PART
decide  POSTag:  VERB
whether  POSTag:  SCONJ
marijuana  POSTag:  NOUN
is  POSTag:  AUX
good  POSTag:  ADJ
or  POSTag:  CCONJ
harmful  POSTag:  ADJ
.  POSTag:  PUNCT
we  POSTag:  PRON
'  POSTag:  PUNCT
re  POSTag:  AUX
focused  POSTag:  VERB
on  POSTag:  SCONJ
whether  POSTag:  SCONJ
the  POSTag:  DET
drug  POSTag:  NOUN
in  POSTag:  ADP
it  POSTag:  PRON
s  POSTag:  PART
natural  POSTag:  ADJ
form  POSTag:  NOUN
is  POSTag:  AUX
``  POSTag:  PUNCT
less  POSTag:  ADV
toxic  POSTag:  ADJ
``  POSTag:  PUNCT
than  POSTag:  ADP
alcohol  POSTag:  NOUN
.  POSTag:  PUNCT
in  POSTag:  ADP
that  POSTag:  DET
regard  POSTag:  NOUN
science  POSTag:  NOUN
and  POSTag:  CCONJ
statistics  POSTag:  NOUN
present  POSTag:  VERB
a  POSTag:  DET
strong  POSTag:  ADJ
case  POSTag: 

...

a  POSTag:  DET
firm  POSTag:  NOUN
hired  POSTag:  VERB
to  POSTag:  PART
do  POSTag:  VERB
common  POSTag:  ADJ
core  POSTag:  NOUN
testing  POSTag:  NOUN
in  POSTag:  ADP
florida  POSTag:  PROPN
will  POSTag:  AUX
``  POSTag:  PART
attract  POSTag:  VERB
every  POSTag:  DET
one  POSTag:  NUM
of  POSTag:  ADP
your  POSTag:  PRON
children  POSTag:  NOUN
to  POSTag:  PART
become  POSTag:  VERB
as  POSTag:  ADV
homosexual  POSTag:  ADJ
as  POSTag:  SCONJ
they  POSTag:  PRON
possibly  POSTag:  ADV
can  POSTag:  AUX
``  POSTag:  PUNCT
van  POSTag:  PROPN
zant  POSTag:  PROPN
said  POSTag:  VERB
.  POSTag:  PUNCT
the  POSTag:  DET
firm  POSTag:  NOUN
hired  POSTag:  VERB
to  POSTag:  PART
do  POSTag:  VERB
testing  POSTag:  NOUN
air  POSTag:  NOUN
separately  POSTag:  ADV
provides  POSTag:  VERB
information  POSTag:  NOUN
to  POSTag:  ADP
entities  POSTag:  NOUN
that  POSTag:  PRON
request  POSTag:  VERB
it  POSTag:  PRON
to  POSTag:  PART
help  POSTag:  VERB
them  POSTag:  PRON
assist  PO

lemieux  POSTag:  PROPN
did  POSTag:  AUX
n't  POSTag:  PART
compare  POSTag:  VERB
rubio  POSTag:  PROPN
and  POSTag:  CCONJ
obama  POSTag:  PROPN
on  POSTag:  ADP
an  POSTag:  DET
issue  POSTag:  NOUN
such  POSTag:  ADJ
as  POSTag:  ADP
those  POSTag:  PRON
listed  POSTag:  VERB
at  POSTag:  ADP
the  POSTag:  DET
start  POSTag:  NOUN
of  POSTag:  ADP
the  POSTag:  DET
ad  POSTag:  NOUN
-  POSTag:  PUNCT
-  POSTag:  PUNCT
he  POSTag:  PRON
said  POSTag:  VERB
they  POSTag:  PRON
both  POSTag:  DET
used  POSTag:  VERB
a  POSTag:  DET
familiar  POSTag:  ADJ
campaign  POSTag:  NOUN
tactic  POSTag:  NOUN
throwing  POSTag:  VERB
``  POSTag:  PUNCT
bombs  POSTag:  NOUN
``  POSTag:  PUNCT
about  POSTag:  ADP
something  POSTag:  PRON
they  POSTag:  PRON
did  POSTag:  AUX
n't  POSTag:  PART
have  POSTag:  VERB
to  POSTag:  PART
vote  POSTag:  VERB
on  POSTag:  ADP
themselves  POSTag:  PRON
.  POSTag:  PUNCT
the  POSTag:  DET
ad  POSTag:  NOUN
provides  POSTag:  VERB
no  POSTag:  DET
explanatio

he  POSTag:  PRON
repeated  POSTag:  VERB
that  POSTag:  DET
statement  POSTag:  NOUN
in  POSTag:  ADP
a  POSTag:  DET
flier  POSTag:  NOUN
that  POSTag:  PRON
he  POSTag:  PRON
sent  POSTag:  VERB
out  POSTag:  ADP
to  POSTag:  ADP
constitutes  POSTag:  NOUN
.  POSTag:  PUNCT
when  POSTag:  ADV
we  POSTag:  PRON
contacted  POSTag:  VERB
ers  POSTag:  NOUN
officials  POSTag:  NOUN
last  POSTag:  ADJ
week  POSTag:  NOUN
they  POSTag:  PRON
told  POSTag:  VERB
us  POSTag:  PRON
they  POSTag:  PRON
had  POSTag:  AUX
``  POSTag:  PUNCT
since  POSTag:  SCONJ
discovered  POSTag:  VERB
there  POSTag:  PRON
is  POSTag:  VERB
another  POSTag:  DET
senator  POSTag:  NOUN
who  POSTag:  PRON
is  POSTag:  AUX
not  POSTag:  PART
a  POSTag:  DET
member  POSTag:  NOUN
of  POSTag:  ADP
lrs  POSTag:  NOUN
.  POSTag:  PUNCT
``  POSTag:  PUNCT
our  POSTag:  PRON
ruling  POSTag:  NOUN
:  POSTag:  PUNCT
balfour  POSTag:  PROPN
’  POSTag:  PART
s  POSTag:  PART
campaign  POSTag:  NOUN
flier  POSTag:  NOUN
sa

hillary  POSTag:  PROPN
clinton  POSTag:  PROPN
said  POSTag:  VERB
she  POSTag:  PRON
and  POSTag:  CCONJ
bill  POSTag:  NOUN
were  POSTag:  AUX
in  POSTag:  ADP
debt  POSTag:  NOUN
and  POSTag:  CCONJ
dead  POSTag:  ADJ
broke  POSTag:  VERB
when  POSTag:  ADV
they  POSTag:  PRON
left  POSTag:  VERB
the  POSTag:  DET
white  POSTag:  ADJ
house  POSTag:  NOUN
.  POSTag:  PUNCT
the  POSTag:  DET
public  POSTag:  ADJ
record  POSTag:  NOUN
shows  POSTag:  VERB
that  POSTag:  SCONJ
they  POSTag:  PRON
possibly  POSTag:  ADV
had  POSTag:  VERB
more  POSTag:  ADJ
liabilities  POSTag:  NOUN
than  POSTag:  ADP
assets  POSTag:  NOUN
but  POSTag:  CCONJ
it  POSTag:  PRON
doesn  POSTag:  VERB
’  POSTag:  PUNCT
t  POSTag:  NOUN
show  POSTag:  VERB
that  POSTag:  PRON
conclusively  POSTag:  ADV
.  POSTag:  PUNCT
more  POSTag:  ADV
important  POSTag:  ADJ
a  POSTag:  DET
balance  POSTag:  NOUN
sheet  POSTag:  NOUN
does  POSTag:  AUX
not  POSTag:  PART
tell  POSTag:  VERB
the  POSTag:  DET
full  POSTa

when  POSTag:  ADV
raddatz  POSTag:  PROPN
said  POSTag:  VERB
nixon  POSTag:  PROPN
hadn  POSTag:  VERB
’  POSTag:  PUNCT
t  POSTag:  PROPN
visited  POSTag:  VERB
the  POSTag:  DET
scene  POSTag:  NOUN
of  POSTag:  ADP
brown  POSTag:  ADJ
’  POSTag:  PUNCT
s  POSTag:  PART
death  POSTag:  NOUN
for  POSTag:  ADP
five  POSTag:  NUM
days  POSTag:  NOUN
nixon  POSTag:  PROPN
replied  POSTag:  VERB
``  POSTag:  PUNCT
i  POSTag:  PRON
've  POSTag:  AUX
been  POSTag:  AUX
here  POSTag:  ADV
almost  POSTag:  ADV
every  POSTag:  DET
day  POSTag:  NOUN
.  POSTag:  PUNCT
``  POSTag:  PUNCT
brown  POSTag:  NOUN
was  POSTag:  AUX
shot  POSTag:  VERB
aug  POSTag:  PROPN
.  POSTag:  PUNCT
9  POSTag:  X
and  POSTag:  CCONJ
violence  POSTag:  NOUN
broke  POSTag:  VERB
out  POSTag:  ADP
aug  POSTag:  PROPN
.  POSTag:  PUNCT
10  POSTag:  X
.  POSTag:  PUNCT
nixon  POSTag:  PROPN
first  POSTag:  ADV
visited  POSTag:  VERB
a  POSTag:  DET
nearby  POSTag:  ADJ
city  POSTag:  NOUN
on  POSTag:  ADP
aug  POST